# How Gauss Determined the Orbit of Ceres

J. Tennenbaum and B. Director, 1998 [[pdf](https://archive.schillerinstitute.com/fidelio_archive/1998/fidv07n02-1998Su/fidv07n02-1998Su_004-how_gauss_determined_the_orbit_o.pdf)]

## Validation with synthetic data

In [ ]:
import numpy as np, matplotlib.pyplot as plt, scipy

norm = np.linalg.norm

π = np.pi

import kepler
from kepler import vec

mu = 0.000296 # UA3 d-2

In [ ]:
p1 = vec(0,1,0)
p2 = vec(1,0,0)

orb_earth  = kepler.Lambert(p1,p2,mu,90)
kepler.info_orbit(orb_earth)

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')
kepler.fullOrbit(ax,orb_earth,"Earth")
sz=2
ax.set_xlim(-sz,sz);
ax.set_ylim(-sz,sz);
ax.set_zlim(-sz,sz);

In [ ]:
c1 = vec(0,2,-.3)
c2 = vec(2,0,-.3)

orb_ceres  = kepler.Lambert(c1,c2,mu,90*2)

a,e = orb_ceres[:2]
p = a*(1-e**2)
ε,h,μ = kepler.physconst(orb_ceres)

print(f'True: a={a:.4f}, e={e:.4f}, p={p:.4f}, h={h:.4f}')
kepler.info_orbit(orb_ceres)


fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')
kepler.fullOrbit(ax,orb_ceres,"Ceres")
sz=6
ax.set_xlim(-sz,sz);
ax.set_ylim(-sz,sz);
ax.set_zlim(-sz,sz);

In [ ]:
# initial offsets
time0 = 20
time1 = 50
# observation times
t_1,t_2,t_3 = Ts = [0,20,40]

# check time
t_4 = 40 + 14*30

path_earth = kepler.mkPath(orb_earth)

[E1,E2,E3,E4] = [ path_earth(time0 + t)[0] for t in Ts + [t_4] ]

E = [E1,E2,E3]

Ebad = [-E1,-E2,-E3]

elip_earth = np.array([path_earth(t)[0] for t in np.linspace(0,orb_earth[-1],300)])

path_ceres = kepler.mkPath(orb_ceres)

[C1,C2,C3,C4] = [ path_ceres(time1 + t)[0] for t in Ts + [t_4] ]

C = [C1,C2,C3] 

D = [kepler.unit(c-e) for e,c in zip(E,C)]

In [ ]:
def show():
    fig = plt.figure(figsize=(8,8))
    ax = fig.add_subplot(111, projection='3d')

    kepler.fullOrbit(ax,orb_ceres,"Ceres")
    ax.plot(*elip_earth.T,color='black',label='Earth')
    ax.plot(*vec(E1,E2,E3).T,'.',color='red',label='observations')
    ax.plot(*vec(C1,C2,C3).T,'.',color='green',label='true depths')
    dl = 3
    for (x,y,z),(X,Y,Z) in zip(D,E):
        ax.plot([X,X+dl*x],[Y,Y+dl*y],[Z,Z+dl*z],color='red',lw=0.3)
    sz = 4
    ax.set_xlim(-sz,sz);
    ax.set_ylim(-sz,sz);
    ax.set_zlim(-sz,sz);
    ax.view_init(elev=25, azim=25)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_zticklabels([])
    plt.legend();
    return ax

show();

In [ ]:
F,P_2,_ = kepler.Gauss0(E,D,Ts)

P_2_2 = P_2
for _ in range(5):
    F2,P_2_2,G = kepler.Gauss0(E,D,Ts, norm(P_2_2))

estimated_depth2 = norm(P_2_2 - E2)
print('Gauss depth2:', estimated_depth2)

EP1, EP2, EP3 = kepler.SolveAreas(E,D,Ts, estimated_depth2)

In [ ]:
ax = show()
ax.plot(*vec(F).T,'.',color='blue',label='F')
ax.plot(*vec(P_2).T,'.',color='cyan',label='initial 2nd depth')
ax.plot(*vec(EP1, EP2, EP3).T,'.',color='black',label='estimated depths');
ax.legend();

Estimation with inverse projection:

In [ ]:
orb_inv = kepler.Orbit3Points(EP1,EP2,EP3,mu)
fun = kepler.mkPath(orb_inv)
estimate_inverse = np.array([fun(t)[0] for t in np.linspace(0,orb_inv[-1],300)])
kepler.info_orbit(orb_inv)
EP4 = fun(time1 + t_4)[0]
EP4t = path_ceres(time1 + t_4)[0]

In [ ]:
kepler.info_orbit(orb_ceres)

Simpler approximation:

In [ ]:
gauss_orbit = kepler.GaussOrbit([EP1,EP2,EP3],Ts)
fun = kepler.mkPath(gauss_orbit)
estimate_gauss = np.array([fun(t)[0] for t in np.linspace(0,gauss_orbit[-1],300)])
kepler.info_orbit(gauss_orbit)

In [ ]:
ax = show()
ax.plot(*vec(EP1, EP2, EP3).T,'.',color='gray',label='estimated depths');
ax.plot(*estimate_gauss.T,color='brown',lw=1,ls='dashed',label='Gauss h')
ax.plot(*estimate_inverse.T,color='brown',lw=1,label='Gauss p')
ax.plot(*vec(E4, EP4, EP4t).T,'.',color='black',label='test');
ax.legend();

## Example with "true" observations 

### Sun

Three positions of the Sun for estimation of Earth's orbit

![sun](https://raw.githubusercontent.com/albertoruiz/jupyterlite/main/data/ceres/sun-jul.png)

![sun](https://raw.githubusercontent.com/albertoruiz/jupyterlite/main/data/ceres/sun-aug.png)

![sun](https://raw.githubusercontent.com/albertoruiz/jupyterlite/main/data/ceres/sun-sep.png)

### Ceres

Three geocentric positions and additional test point:

![ceres](https://raw.githubusercontent.com/albertoruiz/jupyterlite/main/data/ceres/ceres-jul.png)

![ceres](https://raw.githubusercontent.com/albertoruiz/jupyterlite/main/data/ceres/ceres-aug.png)

![ceres](https://raw.githubusercontent.com/albertoruiz/jupyterlite/main/data/ceres/ceres-sep.png)

![ceres](https://raw.githubusercontent.com/albertoruiz/jupyterlite/main/data/ceres/ceres-dec.png)

### Input data

In [ ]:
import numpy as np, matplotlib.pyplot as plt, scipy

norm = np.linalg.norm

π = np.pi

import kepler
from kepler import DIR, RA, DEC, Sun_distance, vec

import julian
import datetime

In [ ]:
t0 = julian.to_jd(datetime.datetime(2020,  1, 1, 0))

t1 = julian.to_jd(datetime.datetime(2020,  7, 11, 14))
t2 = julian.to_jd(datetime.datetime(2020,  8, 11, 14))
t3 = julian.to_jd(datetime.datetime(2020,  9, 11, 14))
t4 = julian.to_jd(datetime.datetime(2020, 12, 11, 14))

In [ ]:
E1 = - DIR( RA(7, 24, 9.4),   DEC(22, 1, 40.2) )* Sun_distance(t1-t0)
E2 = - DIR( RA(9, 25, 58.4),  DEC(15, 6, 11.3) )* Sun_distance(t2-t0)
E3 = - DIR( RA(11, 19, 33.9), DEC(4, 20, 59.4) )* Sun_distance(t3-t0)

E4 = - DIR( RA(17, 15, 23.6), -DEC(23, 2, 25.3) )* Sun_distance(t4-t0)

E = [E1,E2,E3]

In [ ]:
# Ceres directions
D1 = DIR( RA(23, 15, 52.3),  -DEC( 18, 39, 33.6) )
D2 = DIR( RA(23,  6,  0.5),  -DEC( 21, 57, 16.9) )
D3 = DIR( RA(22, 41, 13.2),  -DEC( 24, 52, 30.2) )

D4 = DIR( RA(22, 46, 47.6),  -DEC( 18, 52, 48.7) )

D = [D1,D2,D3]

In [ ]:
Ts = [t1,t2,t3]
print('Observation times (JD)', Ts+[t4])

### Earth

#### From 3D positions

In [ ]:
earth_period = 365.256363004

mu = 0.000296 # UA3 d-2

In [ ]:
orb_earth = kepler.Orbit3Points(E1,E2,E3,mu)
kepler.info_orbit(orb_earth)

path_earth = kepler.mkPathAbsTime(orb_earth,E1,t1)
E_e = [E1_e,E2_e,E3_e,E4_e] = [ path_earth(t)[0] for t in Ts + [t4] ]
dires = kepler.unit(E4_e)
error_angle = np.arccos( dires @ kepler.unit(E4) )
print('Angular error of test point:', kepler.prettyAngle(error_angle))

#### From directions and times

In [ ]:
# Con ángulos y tiempos deducimos la órbita, pero necesitamos período y mu para a
SD = [ kepler.unit(e) for e in [E1,E2,E3] ]

#print( kepler.SolveAreas( np.zeros([3,3]), SD, Ts, 1 ) )
edist = kepler.OptimizeBrute([1*d for d in SD], np.zeros([3,3]), SD, Ts, mu, earth_period)
print([norm(e) for e in edist])
print([norm(e) for e in [E1,E2,E3]] )

orb_earth  = kepler.Orbit3Points(*edist,mu)
kepler.info_orbit(orb_earth)

path_earth = kepler.mkPathAbsTime(orb_earth,edist[0],t1)
E_e = [E1_e,E2_e,E3_e,E4_e] = [ path_earth(t)[0] for t in Ts + [t4] ]
dires = kepler.unit(E4_e)
error_angle = np.arccos( dires @ kepler.unit(E4) )
print('Angular error of test point:', kepler.prettyAngle(error_angle))

### Ceres

The depth of the second point is estimated and refined using Gauss' method:

In [ ]:
F,P_2,_ = kepler.Gauss0(E,D,Ts)

P_2_2 = P_2
for _ in range(4):
    print('---')
    F2,P_2_2,G = kepler.Gauss0(E,D,Ts, norm(P_2_2))

estimated_depth2 = norm(P_2_2 - E2)
print('Gauss depth2:', estimated_depth2)

Then the three depths of Ceres are:

In [ ]:
EP1, EP2, EP3 = kepler.SolveAreas(E,D,Ts, estimated_depth2)

if False:
    print('Optimization')
    EP1, EP2, EP3 = kepler.Optimize3Points([EP1, EP2, EP3], E,D,Ts, mu)
    print('refined depth 2:', norm(EP2[1] - E2) )

The orbit is obtained from the three 3D points:

In [ ]:
print('inverse projection')
orb_ceres = kepler.Orbit3Points(EP1,EP2,EP3,mu)
kepler.info_orbit(orb_ceres)

Finally we check the prediction on the test position:

In [ ]:
fun = kepler.mkPathAbsTime(orb_ceres,EP1,t1)

EP1, EP2, EP3, EP4 = [ fun(t)[0] for t in Ts + [t4] ]

path_ceres = np.array([fun(t)[0] for t in np.linspace(t1,t1+orb_ceres[-1],300)])

dires = kepler.unit(EP4 - E4)
error_angle = np.arccos( dires @ D4 )
print('Angular error of test point:', kepler.prettyAngle(error_angle))

In [ ]:
# interactive version

%pip install ipympl
%matplotlib ipympl

def clean(fig):
    fig.canvas.toolbar_visible = False
    fig.canvas.header_visible = False
    fig.canvas.footer_visible = False
    fig.canvas.capture_scroll = False

def fig3d(size=(6,5)):
    fig = plt.figure(figsize=size)
    plt.subplots_adjust(top=1)
    clean(fig)
    return fig, fig.add_subplot(111, projection='3d')

def show(size=(8,8)):
    fig, ax = fig3d(size=size)

    kepler.fullOrbit(ax,orb_earth,"Earth")

    ax.plot(*vec(E4_e).T,'.',color='green')
    ax.plot(*vec(E1,E2,E3,E4).T,'.',color='red')
    ax.plot(*vec(*E_e).T,'.',color='blue')

    ax.plot(*vec(EP1, EP2, EP3, EP4).T,'.',color='black')

    ax.plot(*path_ceres.T,color='brown',lw=1,label='Ceres')

    ax.plot(*vec(P_2).T,'.',color='gray')

    dl = 3.5
    for (x,y,z),(X,Y,Z) in zip(D+[D4],E+[E4]):
        ax.plot([X,X+dl*x],[Y,Y+dl*y],[Z,Z+dl*z],color='red',lw=0.3)

    sz = 4
    ax.set_xlim(-sz,sz); # ax.set_xlabel('x')
    ax.set_ylim(-sz,sz); #ax.set_ylabel('y')
    ax.set_zlim(-sz,sz); #ax.set_zlabel('z');
    ax.view_init(elev=25, azim=25)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_zticklabels([])
    plt.legend()
    #plt.tight_layout()
    plt.title('Gauss determines the orbit of Ceres')
    #plt.axis('off')
    plt.show()

In [ ]:
show()